In [ ]:
!pip install tensorflow==1.14.0

In [1]:
import tensorflow as tf
import tensorflow_model_analysis as tfma
import numpy as np
import os

print("TF version: {}".format(tf.__version__))

/Users/jiankaiwang/devops/pyenv/tfx/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/jiankaiwang/devops/pyenv/tfx/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/jiankaiwang/devops/pyenv/tfx/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/jiankaiwang/devops/pyenv/tfx/lib/pyt



TF version: 1.14.0


# Brief

In this tutorial, you'll learn how to complete a model training, evaluation, and prediction via `tf.estimator` API.

# Downloading and Processing Dataset

In [3]:
from tensorflow.examples.tutorials.mnist import input_data

In [4]:
if os.path.exists("/Users/jiankaiwang/devops/tmp/MNIST_data/"):
  mnist = input_data.read_data_sets("/Users/jiankaiwang/devops/tmp/MNIST_data/", one_hot=True)
else:
  mnist = input_data.read_data_sets("/tmp/MNIST_data/", one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /Users/jiankaiwang/devops/tmp/MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /Users/jiankaiwang/devops/tmp/MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting /Users/jiankaiwang/devops/tmp/MNIST_data/t10k-images-idx3-ubyte.gz
Extracting /Users/jiankaiwang/devops/tmp/MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


# Building Model and Training

In `tf.Estimator` API, you have to complete both `model_fn` and `params`.
* `model_fn`: define the neural network
* `params`: define the hyperparameters used in training neural network

The following is basic `model_fn` example.
```python
def model_fn(features, labels, mode, params):
   # Logic to do the following:
   # 1. Configure the model via TensorFlow operations
   # 2. Define the loss function for training/evaluation
   # 3. Define the training operation/optimizer
   # 4. Generate predictions
   # 5. Return predictions/loss/train_op/eval_metric_ops in EstimatorSpec object
   return EstimatorSpec(mode, predictions, loss, train_op, eval_metric_ops)
```

In [5]:
def layer(input, weight_shape, bias_shape):
    weight_std = (2.0 / weight_shape[0]) ** 0.5
    w_init = tf.random_normal_initializer(stddev=weight_std)
    b_init = tf.constant_initializer(value=0)
    W = tf.get_variable(name="W", shape=weight_shape, initializer=w_init)
    b = tf.get_variable(name="b", shape=bias_shape, initializer=b_init)
    return tf.nn.relu(tf.matmul(input, W) + b)

In [55]:
def evaluate(output, y):

  compare = tf.equal(tf.argmax(output, axis=1), tf.argmax(y, axis=1))
  compare_to_float = tf.cast(compare, tf.float32)
    
  accuracy, updated_op = tf.metrics.mean(compare_to_float)

  return accuracy, updated_op

In [92]:
def model_fn(features, labels, mode, params):
    
    # build a mlp model
    with tf.variable_scope("hidden1"):
        hidden1 = layer(features["image"], [784, 1024], [1024])
    with tf.variable_scope("hidden2"):
        hidden2 = layer(hidden1, [1024, 512], [512])
    with tf.variable_scope("output"):
        output = layer(hidden2, [512, 10], [10])
    
    # define the result
    result = tf.identity(output, name="class")
    predict_dict = {"class": result}
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(
            mode=mode,
            predictions=predict_dict)
    
    # define loss function
    cross_entropy = tf.nn.softmax_cross_entropy_with_logits_v2(logits=output, labels=labels)
    loss = tf.reduce_mean(cross_entropy)
    
    # define optimizer
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=params["learning_rate"])
    train_opt = optimizer.minimize(loss, global_step=tf.train.get_global_step())
    
    # metrics
    eval_metric_ops = {
        #"accuracy": tf.metrics.accuracy(labels, output)
        "accuracy": evaluate(output, labels)
    }
    
    # return an estimator spec
    return tf.estimator.EstimatorSpec(
        mode=mode,
        loss=loss,
        train_op=train_opt,
        eval_metric_ops=eval_metric_ops)

Start training a model.

In [89]:
batch_size = 32
epochs = mnist.train.num_examples // 32

model_params = {"learning_rate": 1e-4}

In [90]:
nn = tf.estimator.Estimator(model_fn=model_fn, params=model_params)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/var/folders/4b/wh1nthj1563b1gl58ydgwnfw0000gp/T/tmp0ypx7mmn', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x12bd356d8>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [91]:
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"image": mnist.train.images.astype("float32")},
    y=mnist.train.labels.astype("float32"),
    num_epochs=epochs,
    shuffle=True)

# Train
nn.train(input_fn=train_input_fn, steps=200)

INFO:tensorflow:Calling model_fn.
<dtype: 'float32'>
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into /var/folders/4b/wh1nthj1563b1gl58ydgwnfw0000gp/T/tmp0ypx7mmn/model.ckpt.
INFO:tensorflow:loss = 2.376337, step = 1
INFO:tensorflow:global_step/sec: 78.5194
INFO:tensorflow:loss = 2.4521444, step = 101 (1.275 sec)
INFO:tensorflow:Saving checkpoints for 200 into /var/folders/4b/wh1nthj1563b1gl58ydgwnfw0000gp/T/tmp0ypx7mmn/model.ckpt.
INFO:tensorflow:Loss for final step: 2.40667.


# Evaluating the model.

In [82]:
# Score accuracy
test_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"image": mnist.test.images.astype("float32")},
    y=mnist.test.labels.astype("float32"),
    num_epochs=1,
    shuffle=False)

ev = nn.evaluate(input_fn=test_input_fn)
print("Loss: %s" % ev["loss"])
print("Accuracy: %s" % ev["accuracy"])

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-11-05T16:34:19Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /var/folders/4b/wh1nthj1563b1gl58ydgwnfw0000gp/T/tmpz84lwrvd/model.ckpt-2000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2019-11-05-16:34:20
INFO:tensorflow:Saving dict for global step 2000: accuracy = 0.3318, global_step = 2000, loss = 2.1445124
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 2000: /var/folders/4b/wh1nthj1563b1gl58ydgwnfw0000gp/T/tmpz84lwrvd/model.ckpt-2000
Loss: 2.1445124
Accuracy: 0.3318


The following is an example for training and evaluating.

```python
train_spec = tf.estimator.TrainSpec(
                input_fn=generate_input_fn(data_train),
                max_steps=3000)

exporter = tf.estimator.LatestExporter('Servo', serving_input_fn)

eval_spec=tf.estimator.EvalSpec(
            input_fn=generate_input_fn(data_test),
            steps=1,
            exporters=exporter)

tf.estimator.train_and_evaluate(generate_estimator(output_dir), train_spec, eval_spec)
```

# Prediction

In [83]:
# Print out predictions
predict_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"image": mnist.test.images[2:4].astype('float32')},
    num_epochs=1,
    shuffle=False)

predictions = nn.predict(input_fn=predict_input_fn)
for i, p in enumerate(predictions):
  print("Prediction %s: %s" % (i + 1, p["class"]))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /var/folders/4b/wh1nthj1563b1gl58ydgwnfw0000gp/T/tmpz84lwrvd/model.ckpt-2000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
Prediction 1: [0.24514045 0.7282403  0.40821418 0.1869801  0.         0.
 0.         0.         0.3044557  0.        ]
Prediction 2: [0.887169   0.         0.37492734 0.         0.0809705  0.5858083
 0.3675366  0.         0.7051594  0.        ]


# Exporting the SavedModel

A simple example for a `serving_input_fn`.

```python
def serving_input_fn():
  #feature_placeholders are what the caller of the predict() method will have to provide
  feature_placeholders = {
      column.name: tf.placeholder(column.dtype, [None])
      for column in feature_cols
  }
  
  #features are what we actually pass to the estimator
  features = {
    # Inputs are rank 1 so that we can provide scalars to the server
    # but Estimator expects rank 2, so we expand dimension
    key: tf.expand_dims(tensor, -1)
    for key, tensor in feature_placeholders.items()
  }
  return tf.estimator.export.ServingInputReceiver(
    features, feature_placeholders
  )
```

In [13]:
def serving_input_receiver_fn():
    """Serving input_fn that builds features from placeholders

    Returns
    -------
    tf.estimator.export.ServingInputReceiver
    """
    
    content = tf.placeholder(dtype=tf.float32, shape=[None, 784])
    
    # receiver_tensors here can be replaced by `content`
    receiver_tensors = {'image_content': content}
    
    # definition to features must be the same with data input in trainer (tf.estimator.inputs.numpy_input_fn)
    features = {'image': content}
    return tf.estimator.export.ServingInputReceiver(features, receiver_tensors)

serving_input_receiver_fn()

ServingInputReceiver(features={'image': <tf.Tensor 'Placeholder:0' shape=(?, 784) dtype=float32>}, receiver_tensors={'image_content': <tf.Tensor 'Placeholder:0' shape=(?, 784) dtype=float32>}, receiver_tensors_alternatives=None)

In [15]:
if os.path.exists("/Users/jiankaiwang/Desktop/TFE_SavedModel/"):
    nn.export_saved_model("/Users/jiankaiwang/Desktop/TFE_SavedModel/", serving_input_receiver_fn)
else:
    nn.export_saved_model("/tmp/TFE_SavedModel/", serving_input_receiver_fn)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Signatures INCLUDED in export for Classify: None
INFO:tensorflow:Signatures INCLUDED in export for Regress: None
INFO:tensorflow:Signatures INCLUDED in export for Predict: ['serving_default']
INFO:tensorflow:Signatures INCLUDED in export for Train: None
INFO:tensorflow:Signatures INCLUDED in export for Eval: None
INFO:tensorflow:Restoring parameters from /var/folders/4b/wh1nthj1563b1gl58ydgwnfw0000gp/T/tmpp77omimo/model.ckpt-2000
INFO:tensorflow:Assets added to graph.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: /Users/jiankaiwang/Desktop/TFE_SavedModel/temp-b'1572937612'/saved_model.pb


## Checking SavedModel Format

In [18]:
saved_model_path = "/Users/jiankaiwang/Desktop/TFE_SavedModel/1572937612"
assert os.path.exists(saved_model_path), "Exported dir was not found."

In [ ]:
#!saved_model_cli show --dir "/Users/jiankaiwang/Desktop/TFE_SavedModel/1572426418" --tag_set serve --signature_def serving_default

## Predicting via SavedModel format

In [19]:
def show_graph_info(graph):
    """List operations in the frozen model.
    """
    print(graph.signature_def)

In [20]:
# you have to reset the graph if you are going to use the same scope
tf.reset_default_graph()

with tf.Session() as sess:
    imported = tf.saved_model.loader.load(
        export_dir=saved_model_path, 
        sess=sess, 
        tags=['serve'])
    
    show_graph_info(imported)
    
    graph = tf.get_default_graph()
    x = graph.get_tensor_by_name("Placeholder:0")
    y = graph.get_tensor_by_name("class:0")
    data = mnist.test.images[2:4].astype("float32")
    res = sess.run(y, feed_dict={x: data})
    print(res)

Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.loader.load or tf.compat.v1.saved_model.load. There will be a new function for importing SavedModels in Tensorflow 2.0.
INFO:tensorflow:Restoring parameters from /Users/jiankaiwang/Desktop/TFE_SavedModel/1572937612/variables/variables
{'serving_default': inputs {
  key: "image_content"
  value {
    name: "Placeholder:0"
    dtype: DT_FLOAT
    tensor_shape {
      dim {
        size: -1
      }
      dim {
        size: 784
      }
    }
  }
}
outputs {
  key: "class"
  value {
    name: "class:0"
    dtype: DT_FLOAT
    tensor_shape {
      dim {
        size: -1
      }
      dim {
        size: 10
      }
    }
  }
}
method_name: "tensorflow/serving/predict"
}
[[0.         0.47255427 0.24939585 0.27586827 0.         0.
  0.         0.29299074 0.07623385 0.1681642 ]
 [0.         0.03338228 0.07360079 0.25867754 0.2637101  0.
  0.         0.45219374 0.31700